## 기본 예시: 프롬프트 + 모델 + 출력 파서

가장 기본적이고 일반적인 사용 사례는 prompt 템플릿과 모델을 함께 연결하는 것입니다. 이것이 어떻게 작동하는지 보기 위해, 각 나라별 수도를 물어보는 Chain을 생성해 보겠습니다.


In [1]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv()

True

In [2]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install -qU langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("CH01-Basic")

LangSmith 추적을 시작합니다.
[프로젝트명]
CH01-Basic


## 프롬프트 템플릿의 활용

`PromptTemplate`

- 사용자의 입력 변수를 사용하여 완전한 프롬프트 문자열을 만드는 데 사용되는 템플릿입니다
- 사용법
  - `template`: 템플릿 문자열입니다. 이 문자열 내에서 중괄호 `{}`는 변수를 나타냅니다.
  - `input_variables`: 중괄호 안에 들어갈 변수의 이름을 리스트로 정의합니다.

`input_variables`

- input_variables는 PromptTemplate에서 사용되는 변수의 이름을 정의하는 리스트입니다.

In [3]:
from langchain_teddynote.messages import stream_response  # 스트리밍 출력
from langchain_core.prompts import PromptTemplate

`from_template()` 메소드를 사용하여 PromptTemplate 객체 생성


In [4]:
# template 정의
template = "{country}의 수도는 어디인가요?"

# from_template 메소드를 이용하여 PromptTemplate 객체 생성
prompt_template = PromptTemplate.from_template(template)
prompt_template

PromptTemplate(input_variables=['country'], input_types={}, partial_variables={}, template='{country}의 수도는 어디인가요?')

In [ ]:
# python
print("{}+{}={}".format(1, 2, 3))

1+2=3


In [7]:
# prompt 생성
prompt = prompt_template.format(country="대한민국")
prompt

'대한민국의 수도는 어디인가요?'

In [8]:
# prompt 생성
prompt = prompt_template.format(country="미국")
prompt

'미국의 수도는 어디인가요?'

In [9]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(
    model="gpt-4.1-nano",
    temperature=0.1,
)

## Chain 생성

### LCEL(LangChain Expression Language)

![lcel.png](./images/lcel.png)

여기서 우리는 LCEL을 사용하여 다양한 구성 요소를 단일 체인으로 결합합니다

```
chain = prompt | model | output_parser
```

`|` 기호는 [unix 파이프 연산자](<https://en.wikipedia.org/wiki/Pipeline_(Unix)>)와 유사하며, 서로 다른 구성 요소를 연결하고 한 구성 요소의 출력을 다음 구성 요소의 입력으로 전달합니다.

이 체인에서 사용자 입력은 프롬프트 템플릿으로 전달되고, 그런 다음 프롬프트 템플릿 출력은 모델로 전달됩니다. 각 구성 요소를 개별적으로 살펴보면 무슨 일이 일어나고 있는지 이해할 수 있습니다.


In [10]:
# prompt 를 PromptTemplate 객체로 생성합니다.
prompt = PromptTemplate.from_template("{topic} 에 대해 쉽게 설명해주세요.")

model = ChatOpenAI(model="gpt-4.1-nano", temperature=0.1)

chain = prompt | model

### invoke() 호출

- python 딕셔너리 형태로 입력값을 전달합니다.(키: 값)
- invoke() 함수 호출 시, 입력값을 전달합니다.

In [11]:
# input 딕셔너리에 주제를 '인공지능 모델의 학습 원리'으로 설정합니다.
input = {"topic": "인공지능 모델의 학습 원리"}

In [12]:
# prompt 객체와 model 객체를 파이프(|) 연산자로 연결하고 invoke 메서드를 사용하여 input을 전달합니다.
# 이를 통해 AI 모델이 생성한 메시지를 반환합니다.
chain.invoke(input)

AIMessage(content="물론입니다! 인공지능 모델의 학습 원리를 쉽게 설명해 드릴게요.\n\n1. **데이터 수집**: 먼저, 인공지능이 배울 수 있도록 많은 예제(데이터)를 모아요. 예를 들어, 고양이 사진과 고양이 이름이 적힌 목록이 될 수 있어요.\n\n2. **모델 설계**: 그런 다음, 이 데이터를 이해하고 학습할 수 있는 '모델'이라는 수학적 구조를 만듭니다. 이 모델은 사람의 뇌와 비슷하게 정보를 처리하려고 해요.\n\n3. **학습 과정**:\n   - 모델은 처음에는 아무것도 모르기 때문에, 예제 데이터를 보고 예측을 해요.\n   - 예를 들어, 사진이 고양이인지 아닌지 맞추려고 시도하는 거죠.\n   - 그리고 모델이 만든 예측이 실제 정답과 얼마나 다른지(오차)를 계산해요.\n\n4. **오차 줄이기**:\n   - 오차를 줄이기 위해, 모델은 자신의 내부 값(파라미터)을 조금씩 조정해요.\n   - 이 과정을 반복하면서, 점점 더 정확하게 예측할 수 있게 되는 거죠.\n\n5. **반복 학습**:\n   - 여러 번 반복하면서, 모델은 점점 더 좋아지고, 새로운 데이터에 대해서도 잘 맞출 수 있게 돼요.\n\n요약하자면, 인공지능 모델은 많은 데이터를 보고, 예측을 하고, 그 오차를 줄이기 위해 계속 수정하면서 학습하는 과정입니다. 이렇게 해서 점점 더 똑똑해지는 거예요!", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 355, 'prompt_tokens': 22, 'total_tokens': 377, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_t

아래는 스트리밍을 출력하는 예시 입니다.

In [13]:
# 스트리밍 출력을 위한 요청
answer = chain.stream(input)
# 스트리밍 출력
stream_response(answer)

물론입니다! 인공지능 모델의 학습 원리를 쉽게 설명해 드릴게요.

1. **데이터 수집**: 먼저, 인공지능이 배우기 위해 많은 예제(데이터)를 모아요. 예를 들어, 고양이 사진을 많이 보여주는 거죠.

2. **모델 설계**: 그런 다음, 이 데이터를 이해하고 학습할 수 있도록 인공지능의 구조(모델)를 만듭니다. 이 구조는 사람의 뇌와 비슷하게 생각할 수 있어요.

3. **학습 과정**:
   - 인공지능은 처음에는 무작위로 예측하거나 판단을 해요.
   - 그리고 실제 정답(예를 들어, 사진이 고양이인지 아닌지)을 알려줍니다.
   - 인공지능은 자신의 예측이 틀렸다는 것을 알고, 틀린 정도(오차)를 계산해요.
   - 이 오차를 줄이기 위해 내부의 '설정값'이나 '규칙'을 조금씩 조정합니다. 이 과정을 여러 번 반복하면서 점점 더 정확하게 예측하게 되는 거죠.

4. **학습 완료**: 여러 번 반복해서 학습하면, 인공지능은 새로운 사진이 주어졌을 때도 고양이인지 아닌지 잘 맞출 수 있게 돼요.

요약하자면, 인공지능은 많은 데이터를 보고, 틀린 점을 수정하면서 점점 더 똑똑해지는 과정이라고 할 수 있어요!

### 출력파서(Output Parser)


In [14]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

Chain 에 출력파서를 추가합니다.

In [ ]:
# 프롬프트, 모델, 출력 파서를 연결하여 처리 체인을 구성합니다.
chain = prompt | model | output_parser

In [20]:
# chain 객체의 invoke 메서드를 사용하여 input을 전달합니다.
input = {"topic": "인공지능 모델의 학습 원리"}
chain.invoke(input)

"물론입니다! 인공지능 모델의 학습 원리를 쉽게 설명해 드릴게요.\n\n1. **데이터 수집**: 먼저, 인공지능이 배우기 위해 많은 예제(데이터)를 모아요. 예를 들어, 고양이 사진을 많이 보여주는 거죠.\n\n2. **모델 설계**: 그 다음, 이 데이터를 이해하고 학습할 수 있는 '모델'이라는 수학적 구조를 만듭니다. 이 모델은 사람의 뇌와 비슷하게 정보를 처리하려고 해요.\n\n3. **학습 과정**:\n   - 모델은 처음에는 아무것도 몰라요.\n   - 데이터를 보여주면서, 모델이 예측한 결과와 실제 정답(예를 들어, 사진이 고양이인지 아닌지)을 비교해요.\n   - 차이(오차)를 계산하고, 이 오차를 줄이기 위해 모델의 내부 값(파라미터)을 조금씩 조정해요.\n   - 이 과정을 여러 번 반복하면서, 모델은 점점 더 정확하게 예측할 수 있게 돼요.\n\n4. **최적화**: 이때 '최적화 알고리즘'이라는 방법을 사용해서, 오차를 가장 적게 만드는 방향으로 파라미터를 계속 수정해요.\n\n5. **학습 완료**: 여러 번 반복해서 학습하면, 모델은 새로운 데이터에 대해서도 잘 예측할 수 있게 돼요.\n\n쉽게 말해, 인공지능 모델은 많은 예제들을 보여주면서 '이것이 맞는 답이다'라고 배우고, 그 경험을 바탕으로 앞으로 비슷한 문제를 해결하는 방법을 익히는 과정입니다."

In [21]:
# 스트리밍 출력을 위한 요청
answer = chain.stream(input)
# 스트리밍 출력
stream_response(answer)

물론입니다! 인공지능 모델의 학습 원리를 쉽게 설명해 드릴게요.

1. **데이터 수집**: 먼저, 인공지능이 배우기 위해 많은 예시(데이터)를 모아요. 예를 들어, 사진, 글, 숫자 등 다양한 정보를 수집하는 거죠.

2. **모델 설계**: 그 다음, 이 데이터를 이해하고 처리할 수 있도록 인공지능의 '뇌' 역할을 하는 모델을 만듭니다. 이 모델은 여러 층으로 이루어진 신경망(인공신경망)일 수 있어요.

3. **학습 과정**:
   - **예측하기**: 모델은 처음에는 아무것도 몰라서 무작위로 예측을 해요.
   - **오차 계산**: 그리고 실제 정답과 비교해서 얼마나 틀렸는지(오차)를 계산해요.
   - **조정하기**: 오차를 줄이기 위해 모델의 내부 값(가중치)을 조금씩 수정해요. 이 과정을 반복하면서 점점 더 정확한 예측을 하게 되는 거죠.

4. **반복 학습**: 이 과정을 여러 번 반복하면서 모델은 점점 더 많은 데이터를 이해하고, 새로운 데이터에 대해서도 잘 맞추도록 학습됩니다.

5. **완성**: 충분히 학습이 되면, 모델은 새로운 데이터에 대해서도 올바른 예측을 할 수 있게 돼요.

쉽게 말하면, 인공지능은 많은 예시를 보고 '이것이 정답이야'라고 배우면서 점점 더 똑똑해지는 과정입니다!

### 템플릿을 변경하여 적용

- 아래의 프롬프트 내용을 얼마든지 **변경** 하여 테스트 해볼 수 있습니다.
- `model_name` 역시 변경하여 테스트가 가능합니다.

In [22]:
template = """
당신은 영어를 가르치는 10년차 영어 선생님입니다. 주어진 상황에 맞는 영어 회화를 작성해 주세요.
양식은 [FORMAT]을 참고하여 작성해 주세요.

#상황:
{question}

#FORMAT:
- 영어 회화:
- 한글 해석:
"""

# 프롬프트 템플릿을 이용하여 프롬프트를 생성합니다.
prompt = PromptTemplate.from_template(template)

# ChatOpenAI 챗모델을 초기화합니다.
model = ChatOpenAI(model_name="gpt-4.1-nano")

# 문자열 출력 파서를 초기화합니다.
output_parser = StrOutputParser()

In [23]:
# 체인을 구성합니다.
chain = prompt | model | output_parser

In [24]:
# 완성된 Chain을 실행하여 답변을 얻습니다.
print(chain.invoke({"question": "저는 식당에 가서 음식을 주문하고 싶어요"}))

- 영어 회화:  
"Hi, I’d like to order some food, please."  
"Could you please tell me the menu?"  
"I'll have the [dish name], please."  
"Can I get that with no onions?"  

- 한글 해석:  
"안녕하세요, 음식을 주문하고 싶어요."  
"메뉴판 좀 보여주시겠어요?"  
"[요리 이름]으로 할게요."  
"양파 없이 해주실 수 있나요?"


In [25]:
# 완성된 Chain을 실행하여 답변을 얻습니다.
# 스트리밍 출력을 위한 요청
answer = chain.stream({"question": "저는 식당에 가서 음식을 주문하고 싶어요"})
# 스트리밍 출력
stream_response(answer)

- 영어 회화:  
"Hello, I would like to order some food, please."  
"Could I see the menu?"  
"May I get the [specific dish]?"  
"Could I have the bill, please?"  

- 한글 해석:  
"안녕하세요, 음식을 주문하고 싶어요."  
"메뉴를 볼 수 있을까요?"  
"[특정 요리]를 주문할게요."  
"계산서 좀 주세요."

In [26]:
# 이번에는 question 을 '미국에서 피자 주문'으로 설정하여 실행합니다.
# 스트리밍 출력을 위한 요청
answer = chain.stream({"question": "미국에서 피자 주문"})
# 스트리밍 출력
stream_response(answer)

- 영어 회화:
A: Hi, I’d like to order a large pepperoni pizza for delivery, please.  
B: Sure! Would you like any sides or drinks with that?  
A: Yes, please. A bottle of soda and some garlic bread.  
B: Great! Can I have your address and phone number?  
A: Of course. It’s 123 Maple Street, and my number is 555-1234.  
B: Thank you! Your order will be delivered in about 30 minutes.  
A: Thanks very much!

- 한글 해석:
A: 안녕하세요, 피자 한 판(대형 페퍼로니 피자)을 배달 주문하고 싶어요.  
B: 네! 사이드나 음료도 함께 원하시나요?  
A: 네, 탄산음료 한 병과 갈릭 브레드도 주세요.  
B: 알겠습니다! 주소와 전화번호를 알려주시겠어요?  
A: 네, 주소는 123 메이플 스트리트이고, 제 번호는 555-1234입니다.  
B: 감사합니다! 주문하신 피자는 약 30분 후에 배달될 예정입니다.  
A: 정말 고마워요!